In [11]:
import json, pprint

import pandas as pd

import matplotlib.pyplot as plt
import japanize_matplotlib
import networkx as nx
from pyvis.network import Network
%matplotlib inline

In [12]:
import os, sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [13]:
from classes.dependency_analysis import DependencyAnalysis

In [14]:
QUESTION_INPUT_PATH = "../datas/question_sentence/dataset_01.json"
GRAPH_DATA_PATH = "../datas/graphs/string_graph.txt"
GRAPH_IMAGE_PATH = "../datas/graphs/dependency_network.html"

In [15]:
with open(QUESTION_INPUT_PATH, "r") as f:
    question_dict = json.load(f)

pprint.pprint(question_dict, width=150)

[{'answer': ['A = (0, 0, 0)',
             'B = (40, 0, 0)',
             'C = (40, 30, 0)',
             'D = (0, 30, 0)',
             'E = (0, 0, 50)',
             'F = (40, 0, 50)',
             'G = (40, 30, 50)',
             'H = (0, 30, 50)',
             'Segment(A, B)',
             'Segment(B, C)',
             'Segment(C, D)',
             'Segment(D, A)',
             'Segment(E, F)',
             'Segment(F, G)',
             'Segment(G, H)',
             'Segment(H, E)',
             'Segment(A, E)',
             'Segment(B, F)',
             'Segment(C, G)',
             'Segment(D, H)',
             'Segment(D, F)',
             'I = PerpendicularLine(B, Segment(D, F))'],
  'id': 1,
  'impression': ['垂線がなかったけど，PerpendicularLine関数の存在を教えたらいけた．'],
  'normalized': '立体ABCD-EFGHは，AB=40cm，AD=30cm，AE=50cmの直方体である．頂点Dと頂点Fを結び，頂点Bから線分DFに引いた垂線と線分DFとの交点をIとする．線分BIの長さは何cmか．',
  'original': '立体ABCD-EFGHは，AB=40cm，AD=30cm，AE=50cmの直方体である．頂点Dと頂点Fを結び，頂点Bから線分DFに引いた垂線と線分DFとの交点をIとする．線分BIの長さは何

In [16]:
# インスタンスのリストを作成
q_instance_list = []
for i in range(len(question_dict)):
   q1 = DependencyAnalysis(
      document = question_dict[i]["normalized"], 
      model = "ja_ginza_electra"
   )
   # 形態素解析を実行
   q1.analyze()
   q_instance_list.append(q1)
   print(f"Created instance of {i+1}/{len(question_dict)}")

Created instance of 1/10
Created instance of 2/10
Created instance of 3/10
Created instance of 4/10
Created instance of 5/10
Created instance of 6/10
Created instance of 7/10
Created instance of 8/10
Created instance of 9/10
Created instance of 10/10


In [17]:
 for q_instance in q_instance_list:
    tokens_list = []
    for token in q_instance.doc:
        token_data = {
            "text":     token.text,     # テキスト
            "lemma":    token.lemma_,   # レンマ
            "pos":      token.pos_,     # 品詞
            "tag":      token.tag_,     # 品詞詳細
            "dep":      token.dep_,     # 構文従属関係
            "shape":    token.shape_,   # 正書法の特徴(x:文字,d:数値)
            "is_alpha": token.is_alpha, # 文字かどうか
            "is_stop":  token.is_stop   # ストップリストの一部かどうか
        }
        tokens_list.append(token_data)
df_tokens = pd.DataFrame(data = tokens_list , columns = token_data)

# トークンデータフレームはExcelで可視化
df_tokens.to_excel("token_list.xlsx", index=False)

In [18]:
INDEX = 0

# 文節をprint
for span in q_instance_list[INDEX].span_list:
   print(f"{span} | ", end="")
print("") # end=""の回避

pprint.pprint(q_instance_list[INDEX].dep_list)

立体ABCD-EFGHは， | AB=40cm， | AD=30cm， | AE=50cmの | 直方体である． | 頂点Dと | 頂点Fを | 結び， | 頂点Bから | 線分DFに | 引いた | 垂線と | 線分DFとの | 交点を | Iと | する． | 線分BIの | 長さは | 何 | cmか． | 
[('AB=40cm，', 'AD=30cm，'),
 ('AD=30cm，', 'AE=50cmの'),
 ('立体ABCD-EFGHは，', '直方体である．'),
 ('AE=50cmの', '直方体である．'),
 ('頂点Dと', '頂点Fを'),
 ('頂点Fを', '結び，'),
 ('頂点Bから', '引いた'),
 ('線分DFに', '引いた'),
 ('引いた', '垂線と'),
 ('垂線と', '線分DFとの'),
 ('線分DFとの', '交点を'),
 ('結び，', 'する．'),
 ('交点を', 'する．'),
 ('Iと', 'する．'),
 ('線分BIの', '長さは'),
 ('長さは', 'cmか．'),
 ('何', 'cmか．')]


In [19]:
# 係り受けstrリストを，グラフ用外部ファイルに保存
with open(GRAPH_DATA_PATH, "w", encoding = "utf-8") as f:
    f.writelines("# begin end\n")
    for dep in q_instance_list[INDEX].dep_list:
        f.writelines(f"{dep[0]} {dep[1]}\n")

In [20]:
# 有向グラフの作成
G = nx.DiGraph()
# ファイルの読み込み
G = nx.read_edgelist(
    GRAPH_DATA_PATH, 
    nodetype = str, 
    create_using = nx.DiGraph(), 
    encoding = "utf-8"
)

pyvis_G = Network(directed=True)
pyvis_G.from_nx(G)
pyvis_G.show(GRAPH_IMAGE_PATH, notebook=False)

# # グラフの描画(plt使用の場合)
# pos = nx.spring_layout(G)
# nx.draw_networkx(
#     G, 
#     pos,
#     with_labels = True, 
#     alpha=0.5,
#     font_family = "Hiragino Sans",
# )
# # 表示
# plt.figure(figsize=(100,80))
# plt.axis("off")
# plt.show()


../datas/graphs/dependency_network.html
